# Guia Oncodata - Sagemaker

Olá! Este é um guia sobre como vamos usar o Sagemaker no trabalho da Oncodata. Ele pode ser lido diretamente pelo Github, mas também possui trechos executáveis para demonstrar procedimentos que vamos seguir nos notebooks do Sagemaker. Leia-o com atenção.

## Primeiro acesso

Vamos enviar um convite para criar uma conta, que você receberá pelo seu email oncodata.com.br. Crie uma conta através deste convite e logue.
As contas de desenvolvedores da Oncodata estão configuradas para ter acesso somente ao Sagemaker Studio e ao S3 (serviço de armazenamento de arquivos estáticos). A liberação de mais funcionalidades poderá ser analisada no futuro, apenas achamos que apenas esses dois serviços serão suficientes.
A primeira tela após o login virá com o botão de acesso ao Sagemaker. Clique-o e siga para o próximo passo.

## Primeiros passos e explicações sobre o Sagemaker Studio

A nossa principal ferramenta de trabalho vai ser o Sagemaker Studio. Ele consiste em um ambiente com um conjunto de ferramentas para auxiliar os desenvolvedores de Machine Learning. Estamos usando uma versão nova, lançada no final de 2023. Antes disso, o Sagemaker possuia apenas o ambiente de organização de notebooks e arquivos, e cada notebook tinha que ser inicializado em uma instância separada. A versão antiga do Sagemaker virou uma ferramenta ainda presente no Sagemaker novo, chama-se Sagemaker Studio Classic. Não vamos usar essa ferramenta. A ferramenta que vamos usar no dia a dia é o JupyterLab. Ele possui uma interface muito parecida com o Classic, mas possui as seguintes diferenças fundamentais:
* A escolha do tipo de máquina é feita na inicialização do JupyterLab, e todos os cadernos rodam na mesma instância.
* O Kernel, que abrange o sistema com as dependências pré-instaladas, já possui quase tudo que vamos precisar independente da instância escolhida (GPU ou CPU). No Classic existiam muitos tipos de imagens para escolher, o que deixava a inicialização mais lenta. A imagem completa do JupyterLab é mais usada, então deixaram a inicialização dela mais rápida.
* Como tudo roda no mesmo ambiente, podemos abrir terminais para instalar novas dependências ou scripts shell e os notebooks já poderão usar tudo sem precisar reiniciar.
* Podemos usar nos notebooks novos o CodeWhisperer, um assistente de AI para escrever e auxiliar no desenvolvimento.

Sendo assim, a primeira coisa a se fazer no Sagemaker Studio é criar um novo JupyterLab space com as configurações padrões, o nome pode ser oncodata-space ou outro qualquer. Para isso, basta clicar em JupyterLab e no botão "Create JupyterLab space".
Depois de criado, devemos iniciar a execução de uma instância para entrar pela primeira vez:
* Em Instance, escolha inicialmente a menor possível, ml.t3.medium. Depois vamos explicar melhor as opções disponíveis.
* Em Image, escolha a última disponível, SageMaker Distribution 1.4.
* O tamanho mínimo de Storage na nossa configuração é de 50GB, imaginamos que não será necessário mais do que isso.
* Em Lifecycle Configuration, escolher o script que estiver disponível. Ele irá instalar dependências de pip e apt comuns nos nossos projetos. Caso sinta falta de alguma dependência extra, podemos adicioná-la no script.
* Em Attach custom EFS filesystem, basta escolher None.

## Como usar o JupyterLab

O JupyterLab funciona como um computador virtual. Ele possui uma aba de File Browser onde você pode armazenar seus arquivos e repositórios. Eles serão persistidos entre as reinicializações do JupyterLab. A pasta root apresentada é a /home/sagemaker-user da instância, mas outras pastas podem ser usadas, como a /tmp ou a /opt.

A instalação de novas dependências pode ser feita tanto pelos notebooks com !pip install, quanto pelo terminal, que pode ser aberto pelo botão de +.

A sessão do sagemaker expira em cerca de 6 horas. Quando isso acontece, os notebooks que estavam ligados continuam como estavam, nada é perdido. Para retornar, basta clicar em save session e sign in, depois clicar em sign in e aparecerá uma popup de login. Ao logar, a popup redireciona para a home do sagemaker. Depois disso, basta fechar a popup e atualizar a página do jupyterLab.

### Escolha de tipo de instância

O uso de instâncias com GPU é cobrado por hora e é bem caro. Todos teremos acesso a GPUs de última geração, como A100 e V100, mas devemos usá-las com moderação.
Antes de ter certeza de que um código está pronto para rodar em GPU, devemos testá-la em máquinas mais baratas e sem GPU. Recomendamos usar a ml.t3.medium nessa etapa. O código deve funcionar de preferência sem modificações, tanto na GPU quanto na CPU. Para isso, basta seguir o modelo abaixo:



In [3]:
# Inicie uma variável que guarda o tipo de dispositivo
import torch
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

from torchvision.models import resnet50
feat_gen = resnet50()
feat_gen = feat_gen.to(device)  # Use o método to(device) para colocar o modelo em GPU ou CPU

A instância ml.t3.medium pode ficar ligada por longos períodos sem custo alto (apenas 5 centavos de dólar por hora), mas lembre de desligá-la se for passar longos períodos sem acessá-la. Porém, **instâncias com GPU devem ser usadas com muito cuidado**, pois podem gerar custos enormes para a Oncodata se forem esquecidas. Estamos em período de experimentação e não sabemos tão bem quais devem ser os tamanhos de instância ideais para cada tarefa. Frequentemente será suficiente usar a máquina com GPU mais barata, ligada por longos períodos, para executar treinos ou geração de features mais comuns. Máquinas com múltiplas GPUs também estão disponíveis e precisam ser usadas com ainda mais cuidado, pois, dependendo da implementação, é possível que apenas uma GPU seja usada, o que seria um desperdício.
A tabela abaixo mostra as instâncias com GPU que temos disponíveis para uso:

| Instance Type    | GPUs                         | vCPUs | Memory (RAM) | Cost (us-east-1)   |
|------------------|------------------------------|-------|--------------|-------------------|
| ml.g4dn.xlarge   | NVIDIA T4 Tensor Core (1x)   | 4     | 16 GB        | \$0.74 per hour  |
| ml.g5.xlarge     | NVIDIA A100 Tensor Core (1x) | 4     | 16 GB        | \$1.41 per hour   |
| ml.g5.4xlarge    | NVIDIA A100 Tensor Core (1x) | 16    | 64 GB        | \$2.03 per hour   |
| ml.p3.2xlarge    | NVIDIA V100 Tensor Core (1x) | 8     | 61 GB        | \$3.83 per hour   |
| ml.g4dn.12xlarge | NVIDIA T4 Tensor Core (4x)   | 48    | 192 GB       | \$4.89 per hour   |
| ml.p3.8xlarge    | NVIDIA V100 Tensor Core (4x) | 32    | 244 GB       | \$14.69 per hour  |
| ml.p3.16xlarge   | NVIDIA V100 Tensor Core (8x) | 64    | 488 GB       | \$28.15 per hour  |
| ml.p3dn.24xlarge | NVIDIA V100 Tensor Core (8x) | 96    | 768 GB       | \$35.89 per hour  |
| ml.p4d.24xlarge  | NVIDIA A100 Tensor Core (8x) | 96    | 1.1 TB       | \$37.69 per hour  |

### Como usar os dados do S3

Uma grande diferença entre o desenvolvimento local e o remoto é que no remoto não temos todos os dados disponíveis na mesma instância. Como podemos usar quantidades enormes de dados, que são armazenados no S3, da ordem de centenas de GB, não fica barato armazenar todos esses dados na instância que estamos trabalhando. Também não conseguimos usar ferramentas para simular o S3 como um driver conectado à instância, pois isso exigiria permissões que não possuimos. Assim, recomendamos utilizar um pequeno utilitário disponível neste repositório para gerenciar os downloads e uploads dos dados. Este utilitário, o S3FileManager, realiza as seguintes tarefas:

* Cria a estrutura de arquivos do S3 em uma pasta local com arquivos vazios
* Baixa arquivos para uso por demanda
* Remove arquivos que não estão sendo utilizados caso o espaço disponível não seja suficiente para baixar o próximo arquivo
* Envia arquivos locais para o S3, mantendo a estrutura local, por demanda

Para usar, siga o seguinte exemplo:

In [13]:
import os
import slideio
from utils.S3FileManager import S3FileManager, S3UploadSync

bucket_name = 'oncodata-datasources/tcga/lung/'  # A referência de bucket_name pode incluir o caminho de uma pasta do S3, mas comece sempre pelo nome do bucket
local_dir = '/tmp/train-data'  # O local_dir é a pasta local onde a estrutura do S3 será replicada

file_manager = S3FileManager(bucket_name=bucket_name, local_dir=local_dir)  # Instancie o S3FileManager, ele irá criar a estrutura de arquivos. Apenas uma instância de S3FileManager é suportada simultaneamente.
# Agora basta usar os arquivos que estão no local_dir normalmente. O file_manager cuida de detectar se um arquivo está sendo aberto e o baixa automaticamente.
# Por exemplo, pode-se usar:
file_path = '/tmp/train-data/luad/001177f0-b928-472a-93b1-b356ac3529d1/TCGA-55-8507-01A-01-TS1.f6baa937-6390-4b2f-916b-705be1bb8926.svs'
slide_handler = slideio.open_slide(file_path, 'SVS')

print(os.path.getsize(file_path))  # Printando o tamanho do arquivo baixado, para provar que foi baixado

# Para fazer upload, basta usar o S3UploadSync:
bucket = 'oncodata-sagemaker-shared'  # Apenas o nome do bucket, vamos usar esse bucket como padrão para a maior parte do arquivos gerados por nós
local_dir = '/opt/ml/checkpoints'  # O local_dir é a pasta local que será enviada para o S3 com toda a sua estrutura original
bucket_key = 'ed/checkpoints/stomach/Stomach_feats_mocov3/'  # bucket_key é a pasta do bucket que receberá os arquivos. Mais detalhes em "Organização do S3".
s3Syncher = S3UploadSync(bucket, local_dir, bucket_key)
s3Syncher.sync()

80910155
Uploaded 0 files to oncodata-sagemaker-shared


### Como usar o CodeWhisperer

A instalação do CodeWhisperer já foi habilitada em todos os nossos notebooks. Assim, para usar, basta escrever código e, durante a escrita, o CodeWhisperer irá dar sugestões de código, que podem ser aceitas apertando tab. Uma maneira de invocar manualmente o assistente do CodeWhisperer é escrevendo o que se deseja em um comentário e apertando enter. Outra maneira é usando a tecla de atalho alt+c. Use e abuse ;-)

### Como clonar um repositório privado

Os nossos repositórios no Github são todos privados, então precisamos configurar permissões para que possamos cloná-los. Para isso, preparamos um script para ajudar nesse processo, que envolve a geração de um SSH Key Pair e a inserção dele nas configurações pessoais do Github de cada um. Para executar, basta abrir um terminal e executar `. setup_github.sh`.


## Organização do S3

Estamos organizando os nossos dados em 2 buckets principais no S3. O primeiro é o `oncodata-datasources`. Nele estaremos armazenando dados de treino não modificados que baixamos da Internet, como o TCGA. Vamos seguir uma hierarquia lógica de pastas, separando por fonte, tipo e classificações.

O segundo bucket é o `oncodata-sagemaker-shared`. Nele vamos guardar arquivos gerados por nós, como modelos, features, checkpoints, patches, planilhas, etc. Cada membro terá uma pasta na raiz para usar como quiser.

## Processamento avançado

Grande parte dos processos que vamos fazer poderão ser executados diretamente nos notebooks. Porém, alguns casos necessitarão de mais recursos e tempos imprevisíveis para terminar. Outra eventual necessidade vai ser a de acessar arquivos grandes repetidamente, o que implicaria em altos custos de transferência do S3. Para esses casos, podemos utilizar os training jobs do sagemaker. Seguem abaixo as vantagens de usar esse recurso:
* É possível usar instâncias spot, que são até 53% mais baratas que as que usamos nos notebooks.
* As máquinas são desligadas automaticamente.
* A transferência de dados do S3 é mais rápida, via streaming.
* Os logs no console podem ser acompanhados em tempo real.

As desvantagens são as seguintes:
* Maior complexidade para configurar os scripts.
* O script de treino tem que ser adaptado para este uso.
* O treino para e deve ser recomeçado se houver erro durante o processamento.
* Difícil de debugar possíveis problemas.

Para usar um training job, basta seguir o modelo em estimator_dsmil.ipynb, que referencia o script de treino dsmil_train_job.py. Ele possui comentários para explicar o que deve ser editado.
O código usado nos notebooks será bem próximo ao código usado nos training jobs. As principais diferenças serão:
* O modo de acessar os dados será diferente. Nos notebooks, recomendamos usar o S3FileManager. Nos training jobs, acessamos como arquivos locais em uma localização passada por variáveis de ambiente (args.train). Essa variável será a referência local aos dados do S3 apontados no método estimator.fit.
* Ao usar dataloaders, deve-se tomar cuidado no caso de haver múltiplos nodes, para que a distribuição dos dados entre as GPUs seja bem dividida. Este caso está sendo trado no código de exemplo.
* Na hora de salvar dados, também deve-se tomar cuidado no caso de múltiplos nodes, pois apenas um node deve ser responsável por isso, para não salvar as mesmas coisas várias vezes.

Os resultados dos treinos serão salvos automaticamente no S3 no bucket indicado, mas recomendamos salvar no S3 manualmente os resultados, para que eles já fiquem no lugar adequado e não compactados.
Exemplos e referências para aprender mais podem ser vistas nos seguintes links:
* https://sagemaker-examples.readthedocs.io/en/latest/index.html
* https://github.com/aws-samples/amazon-sagemaker-cv
* https://docs.aws.amazon.com/sagemaker/latest/dg/distributed-training-notebook-examples.html
* https://towardsdatascience.com/amazon-sagemaker-fast-file-mode-d12829479c39

### Informações técnicas relevantes

Em um cenário típico de treinamento distribuído no Amazon SageMaker:

O "world_size" é definido para especificar quantas instâncias (ou nós) de treinamento devem ser provisionadas para executar o treinamento distribuído.
Cada instância de treinamento recebe um "rank" exclusivo, que é usado para identificá-la e coordenar a comunicação entre as instâncias durante o treinamento.
O framework de machine learning distribuído (por exemplo, TensorFlow, PyTorch) geralmente possui funcionalidades embutidas para lidar com a distribuição do treinamento com base nos parâmetros "world_size" e "rank", permitindo que o treinamento seja executado de forma eficiente e coordenada em várias instâncias.

Se você está usando uma única instância que possui múltiplas GPUs, como a instância ml.g4dn.12xlarge no Amazon SageMaker, você geralmente não precisa se preocupar com os parâmetros world_size e rank.

Esses parâmetros são mais relevantes quando você está lidando com treinamento distribuído em várias instâncias, onde cada instância executa um processo de treinamento separado. Em uma instância com múltiplas GPUs, o treinamento é executado localmente em uma única instância, e as GPUs estão disponíveis para o treinamento dentro do mesmo processo. Portanto, não há necessidade de coordenar a comunicação entre instâncias ou atribuir identificadores de "rank" exclusivos.

Ao utilizar uma instância com várias GPUs, como a ml.g4dn.12xlarge, você pode aproveitar todas as GPUs disponíveis para treinar seu modelo de forma eficiente em paralelo. O Amazon SageMaker fornece integração direta com as instâncias com várias GPUs, permitindo que você aproveite todo o poder computacional oferecido por essas instâncias sem se preocupar com a complexidade de lidar com a distribuição do treinamento entre várias instâncias.